# writers

The Midgard **writers** module can be used for writing files. For example following file formats are available via Midgard:

- Bernese station information file in *.STA format
- GipsyX site information file
- GAMIT station information file
- ...

All available writers can be shown with `names` method:

In [ ]:
# Import writers package
from midgard import writers

# List all available writers
writers.names()

## Use of station information file writers 

Following station information file writers exists:

| Writer name        | Description |
|:------------------ | :-----------|
| bernese_abb        | Bernese station abbreviation table in *.ABB format |
| bernese_clu        | Bernese station list file in *.CLU format |
| bernese_crd        | Bernese station coordinate file in *.CRD format | 
| bernese_sta        | Bernese station information file in *.STA format |
| bernese_vel        | Bernese station velocity file in *.VEL format |
| gamit_apr_eq       | GAMIT station coordinates files in *.apr and *.eq format |
| gamit_station_info | GAMIT station information file |
| gipsyx_site_info   | GipsyX station information file |

As a first step the input data has to be generated for the writers. That means station information has to be collected from one or more of the following sources:

| Source             | Parser name | Description |
|:------------------ | :-----------| :-----------|
| SINEX              | sinex_site  | Station information given in SINEX format files, which can inlude information about antenna, receiver, eccentricity and station coordinates (like https://files.igs.org/pub/station/general/igs.snx) | 
| SSC                | ssc_site    |Station coordinate and velocity is given in SSC format files (like https://itrf.ign.fr/docs/solutions/itrf2014/ITRF2014_GNSS.SSC.txt) |
| M3G                | m3g         | M3G application programming interface (API) from https://gnss-metadata.eu/site/api-docs. The M3G is a metadata management and distribution system for multiple GNSS networks (https://gnss-metadata.eu), which is accessable via M3G application programming interface (API).  |

Midgard **parsers** can be used to read SSC/SINEX files and the **site_info** module to access the M3G API. In the following a SSC and a SINEX file are read as **source_data**. These **source_data** are applied to initialize a Midgard **site_info** object. Midgard **site_info** class provides a data structure to save relevant station information. 

In [49]:
# Standard library import
from datetime import datetime
from pprint import pprint # Used to print variables more pretty

# Import Midgard parsers
from midgard import parsers

# Import SiteCoord class
from midgard.site_info.site_info import SiteInfo

# Read station information (antenna, receiver, eccentricity) from SINEX file
p_snx = parsers.parse_file(
            parser_name="sinex_site", 
            file_path="../tests/parsers/example_files/sinex_site",
)

# Read station coordinates/velocities from SSC file
p_ssc = parsers.parse_file(
            parser_name="ssc_site", 
            file_path="../tests/parsers/example_files/ssc_site",
)

# Get station dictionary of SiteInfo instance for SINEX source
site_info = SiteInfo.get(
            source="snx", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_snx.as_dict(),
            stations=p_snx.as_dict().keys(), # List with station identifiers
            source_path=p_snx.file_path,
)

# Get station dictionary of SiteInfo instance for SSC source
site_info_ssc = SiteInfo.get(
            source="ssc", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_ssc.as_dict(),
            stations=p_ssc.as_dict().keys(), # List with station identifiers
            source_path=p_ssc.file_path,
)

# Show site_info dictionary
pprint(site_info)

{'brux': {'antenna': AntennaHistorySinex(station='brux'),
          'eccentricity': EccentricityHistorySinex(station='brux'),
          'identifier': IdentifierSinex(station='brux', ),
          'receiver': ReceiverHistorySinex(station='brux'),
          'site_coord': SiteCoordHistorySinex(station='brux')},
 'pots': {'antenna': AntennaHistorySinex(station='pots'),
          'eccentricity': EccentricityHistorySinex(station='pots'),
          'identifier': IdentifierSinex(station='pots', ),
          'receiver': ReceiverHistorySinex(station='pots'),
          'site_coord': SiteCoordHistorySinex(station='pots')},
 'zimm': {'antenna': AntennaHistorySinex(station='zimm'),
          'eccentricity': EccentricityHistorySinex(station='zimm'),
          'identifier': IdentifierSinex(station='zimm', ),
          'receiver': ReceiverHistorySinex(station='zimm'),
          'site_coord': SiteCoordHistorySinex(station='zimm')}}


The SINEX files like **igs.snx** or **euref.snx** include only approximated station coordinates. Therefore we read in addition the SSC file with precise coordinates and velocities. The SINEX **site_info** dictionary will now be updated with the **site_coord** objects of the SSC **site_info_ssc** dictionary.

In [51]:
# Update SINEX source with station coordinates/velocities from SSC source
for sta in site_info.keys():
    site_info[sta]["site_coord"] = site_info_ssc[sta]["site_coord"]
    
# Show site_info dictionary
pprint(site_info)

{'brux': {'antenna': AntennaHistorySinex(station='brux'),
          'eccentricity': EccentricityHistorySinex(station='brux'),
          'identifier': IdentifierSinex(station='brux', ),
          'receiver': ReceiverHistorySinex(station='brux'),
          'site_coord': SiteCoordHistorySsc(station='brux')},
 'pots': {'antenna': AntennaHistorySinex(station='pots'),
          'eccentricity': EccentricityHistorySinex(station='pots'),
          'identifier': IdentifierSinex(station='pots', ),
          'receiver': ReceiverHistorySinex(station='pots'),
          'site_coord': SiteCoordHistorySsc(station='pots')},
 'zimm': {'antenna': AntennaHistorySinex(station='zimm'),
          'eccentricity': EccentricityHistorySinex(station='zimm'),
          'identifier': IdentifierSinex(station='zimm', ),
          'receiver': ReceiverHistorySinex(station='zimm'),
          'site_coord': SiteCoordHistorySsc(station='zimm')}}


The last step is to use the **site_info** dictionary for writing the station information in different formats with the writers like **bernese_crd**, **bernese_sta**, **gamit_station_info** or **gipsyx_site_info**.

In [ ]:
# Import writer
from midgard.writers.bernese_crd import bernese_crd
from midgard.writers.bernese_sta import bernese_sta
from midgard.writers.gamit_station_info import gamit_station_info
from midgard.writers.gipsyx_site_info import gipsyx_site_info

# Use bernese_crd writer
bernese_crd(
    file_path="./examples/writers/BERNESE.CRD",
    site_info=site_info,
    datum="IGb14", # Reference system of station coordinates
    epoch=datetime(2010,1,1), # Reference epoch of station coordinates
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.CRD', language='python')

In [ ]:
# Use bernese_vel writer
bernese_vel(
    file_path="./examples/writers/BERNESE.VEL",
    site_info=site_info,
    datum="IGb14", # Reference system of station velocities
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.VEL', language='python')

## Use writers module with bernese_sta writer

An example is shown, how to use the **writers** module. As a first step the input data has to be generated, in this example for the **bernese_sta** writer. The **bernese_sta** writer needs station information as input, which we generate by using the M3G application programming interface (API) from https://gnss-metadata.eu/site/api-docs. The M3G is a metadata management and distribution system for multiple GNSS networks (https://gnss-metadata.eu), which is accessable via M3G application programming interface (API). The M3G API is accessed via the Midgard **site_info.m3g** wrapper (see also at the **site_info** Jupyter notebook for more information).

In [ ]:
# Standard library import
from datetime import datetime

# Import M3G API wrapper
from midgard.site_info import m3g

# Import SiteCoord class
from midgard.site_info.site_info import SiteInfo

# Get instance of M3gApi class
api = m3g.api.M3gApi()

# Get sitelog information from BRUX00BEL station in M3G database
source_data = dict()
m3g_brux = api.get_sitelog(filter={"id":{"like":"BRUX00BEL"}})[0]
source_data["BRUX"] = {"BRUX00BEL": m3g_brux}

As a next step the generated **source data** are applied to initialize a Midgard **site_info** object. Midgard **site_info** class provides a data structure to save relevant station information. 

In [ ]:
# Get station dictionary of SiteInfo instance 
site_info = SiteInfo.get(
            source="m3g", 
            source_data=source_data,
            stations="brux",
            source_path=api.url,
)

The last step is to use the **site_info** object for writing the Bernese station information file via the **bernese_sta** writer.

In [ ]:
# Standard library import
from datetime import datetime

# Import Midgard parsers
from midgard import parsers

# Import SiteCoord class
from midgard.site_info.site_info import SiteInfo

# Read station information SINEX file
p = parsers.parse_file(
            parser_name="sinex_site", 
            file_path="../tests/parsers/example_files/sinex_site",
)

# Get station dictionary of SiteInfo instance 
site_info = SiteInfo.get(
            source="snx", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p.as_dict(),
            stations=p.as_dict().keys(), # List with station identifiers
            source_path=p.file_path,
)

In [ ]:
# Import writer
from midgard.writers.bernese_sta import bernese_sta

# Use bernese_sta writer
bernese_sta(
    file_path="./examples/writers/BERNESE.STA",
    site_info=site_info,
    agency="NMA", # Agency which uses this file for processing
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.STA', language='python')